In [2]:
import pandas as pd 
import numpy as np 
import streamlit as st 
from PIL import Image 
from numpy import loadtxt
from xgboost import XGBClassifier
import urllib.request
#Colocamos el lugar de donde extraer el csv
data_location= "https://raw.githubusercontent.com/mwolinsky/Ranking_predictor/main/england-premier-league-players-2018-to-2019-stats.csv"

#Leemos el csv con lo jugadores como indice
df= pd.read_csv(data_location)

#Comenzamos con la creación de la variable target, cuyo valor serán los mejores 15 jugadores de cada posición

df['top_mid_15']=df.rank_in_league_top_midfielders.apply(lambda x: 1 if x>0 and x<=15 else 0)
df['top_def_15']=df.rank_in_league_top_defenders.apply(lambda x: 1 if x>0 and x<=15 else 0)
df['top_att_15']=df.rank_in_league_top_attackers.apply(lambda x: 1 if x>0 and x<=15 else 0)
df['top_15']= df.apply(lambda x: 1 if x.top_mid_15==1 or x.top_def_15==1 or x.top_att_15==1 else 0,axis=1)
df=df.drop(columns=['top_mid_15', 'top_att_15', 'top_def_15'])

df['top_rank']=df.loc[:,['rank_in_league_top_attackers','rank_in_league_top_midfielders','rank_in_league_top_defenders']].apply(lambda x: x.min(),axis=1)


categorical_columns=['position','Current Club','nationality']
for column in categorical_columns:
    dummies = pd.get_dummies(df[column], prefix=column,drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(columns=column)

X=df.loc[:,['goals_per_90_overall', 'assists_per_90_overall', 'goals_involved_per_90_overall', 'min_per_conceded_overall', 'minutes_played_overall']]
y= df.top_15

from sklearn.model_selection import train_test_split

#Con estratificación en y

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state=162)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)

xgb = XGBClassifier(subsample= 0.8999999999999999,n_estimators=100, max_depth=6, learning_rate= 0.1,colsample_bytree= 0.6, colsample_bylevel= 0.7999999999999999)
fit_model=xgb.fit(X_train, y_train)


fit_model.save_model("xgb_model.json")

[21:49:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Maty\anaconda3\envs\dhdsblend2021\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
